<a href="https://colab.research.google.com/github/cloud-annotations/google-colab-training/blob/master/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
The only thing you **NEED** to change is `ANNOTATIONS_FOLDER`.

You can easily create your own annotations for free at [Cloud Annotations](https://cloud.annotations.ai).

### Steps
1. Start a new project
1. Choose **`localization`** as the project type
1. Label your images and videos
1. Export an annotation folder by clicking `File > Export as Create ML`
1. Unzip the folder and upload it to Google Drive


In [0]:
################################################################################
# Things to change:
ANNOTATIONS_FOLDER = 'THE_NAME_OF_YOUR_FOLDER_IN_GOOGLE_DRIVE'
NUM_TRAIN_STEPS = 500
################################################################################

import os
GOOGLE_DRIVE_MOUNT    = '/content/drive'
ANNOTATIONS_PATH      = os.path.join(GOOGLE_DRIVE_MOUNT, 'My Drive', ANNOTATIONS_FOLDER)
ANNOTATIONS_JSON_PATH = os.path.join(ANNOTATIONS_PATH, 'annotations.json')

CHECKPOINT_PATH = '/content/checkpoint'
OUTPUT_PATH     = '/content/output'
EXPORTED_PATH   = '/content/exported'
DATA_PATH       = '/content/data'

LABEL_MAP_PATH    = os.path.join(DATA_PATH, 'label_map.pbtxt')
TRAIN_RECORD_PATH = os.path.join(DATA_PATH, 'train.record')
VAL_RECORD_PATH   = os.path.join(DATA_PATH, 'val.record')

# Install the TensorFlow Object Detection API
In order to use the TensorFlow Object Detection API, we need to clone it's GitHub Repo.

### Dependencies
Most of the dependencies required come preloaded in Google Colab. The only additional package we need to install is TensorFlow.js, which is used for converting our trained model to a model that is compatible for the web.

### Protocol Buffers
The TensorFlow Object Detection API relies on what are called `protocol buffers` (also known as `protobufs`). Protobufs are a language neutral way to describe information. That means you can write a protobuf once and then compile it to be used with other languages, like Python, Java or C.

The `protoc` command used below is compiling all the protocol buffers in the `object_detection/protos` folder for Python.

### Environment
To use the object detection api we need to add it to our `PYTHONPATH` along with `slim` which contains code for training and evaluating several widely used Convolutional Neural Network (CNN) image classification models.

In [21]:
import os

!cd /content
!git clone https://github.com/tensorflow/models.git
!pip install --no-deps tensorflowjs==1.4.0

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

pwd = os.getcwd()
os.environ['PYTHONPATH'] += ':%s:%s/slim' % (pwd, pwd)

fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
/env/python:/content/models/research/:/content/models/research/slim/:`pwd`:`pwd`/slim/:/content/models/research:/content/models/research/slim/:/content/models/research:/content/models/research/slim/:/content/models/research:/content/models/research/slim
/content/models/research


# Test the setup
If everything was set up properly and nothing went wrong, we should be able to run this command.

In [0]:
!python object_detection/builders/model_builder_test.py

# Mount Google Drive
In order to use files from Google Drive we need to mount it to Colab.

When running this command for the first time it will ask you to open a link to accept permissions. After doing so, it will give you an authorization code that you can copy and paste below to complete the mounting process.

In [3]:
from google.colab import drive
drive.mount(GOOGLE_DRIVE_MOUNT)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Generate a Label Map

In [0]:
import os
import json

# Get a list of labels from the annotations.json
labels = {}
with open(ANNOTATIONS_JSON_PATH) as f:
  annotations = json.load(f)
  labels = {l['label'] for a in annotations for l in a['annotations']}

# Create a file named label_map.pbtxt
os.makedirs(DATA_PATH, exist_ok=True)
with open(LABEL_MAP_PATH, 'w') as f:
  # Loop through all of the labels and write each label to the file with an id
  for idx, label in enumerate(labels):
    f.write('item {\n')
    f.write("\tname: '{}'\n".format(label))
    f.write('\tid: {}\n'.format(idx + 1)) # indexes must start at 1
    f.write('}\n')

## Generate the TFRecords


In [5]:
import os
import io
import json
import random

import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

def create_tf_record(annotations, label_map, image_path, output):
  # Create a train.record TFRecord file.
  with tf.python_io.TFRecordWriter(output) as writer:
    # Loop through all the training examples.
    for annotation in annotations:
      try:
        # Make sure the image is actually a file
        img_path = os.path.join(image_path, annotation['image'])   
        if not os.path.isfile(img_path):
          continue
          
        # Read in the image.
        with tf.gfile.GFile(img_path, 'rb') as fid:
          encoded_jpg = fid.read()

        # Open the image with PIL so we can check that it's a jpeg and get the image
        # dimensions.
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = PIL.Image.open(encoded_jpg_io)
        if image.format != 'JPEG':
          raise ValueError('Image format not JPEG')

        width, height = image.size

        # Initialize all the arrays.
        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes = []

        # The class text is the label name and the class is the id. If there are 3
        # cats in the image and 1 dog, it may look something like this:
        # classes_text = ['Cat', 'Cat', 'Dog', 'Cat']
        # classes      = [  1  ,   1  ,   2  ,   1  ]

        # For each image, loop through all the annotations and append their values.
        for a in annotation['annotations']:
          coord = a['coordinates']
          xmin = coord['x'] - (coord['width'] / 2)
          xmax = xmin + coord['width']
          ymin = coord['y'] - (coord['height'] / 2)
          ymax = ymin + coord['height']
          xmins.append(max(xmin / width, 0))
          xmaxs.append(min(xmax / width, 1))
          ymins.append(max(ymin / height, 0))
          ymaxs.append(min(ymax / height, 1))
          label = a['label']
          classes_text.append(label.encode('utf8'))
          classes.append(label_map[label])
      
        # Create the TFExample.
        tf_example = tf.train.Example(features=tf.train.Features(feature={
          'image/height': dataset_util.int64_feature(height),
          'image/width': dataset_util.int64_feature(width),
          'image/filename': dataset_util.bytes_feature(annotation['image'].encode('utf8')),
          'image/source_id': dataset_util.bytes_feature(annotation['image'].encode('utf8')),
          'image/encoded': dataset_util.bytes_feature(encoded_jpg),
          'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
          'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
          'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
          'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
          'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
          'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
          'image/object/class/label': dataset_util.int64_list_feature(classes),
        }))
        if tf_example:
          # Write the TFExample to the TFRecord.
          writer.write(tf_example.SerializeToString())
      except ValueError:
        print('Invalid example, ignoring.')
        pass
      except IOError:
        # print("Can't read example, ignoring.")
        pass

with open(ANNOTATIONS_JSON_PATH) as f:
  annotations = json.load(f)
  # Load the label map we created.
  label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)

  random.seed(42)
  random.shuffle(annotations)
  num_train = int(0.7 * len(annotations))
  train_examples = annotations[:num_train]
  val_examples = annotations[num_train:]

  create_tf_record(train_examples, label_map, ANNOTATIONS_PATH, TRAIN_RECORD_PATH)
  create_tf_record(val_examples, label_map, ANNOTATIONS_PATH, VAL_RECORD_PATH)


Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't read example, ignoring.
Can't rea

## Download a base model

In [0]:
import os
import tarfile

import six.moves.urllib as urllib

# Base models can be found here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
download_base = 'http://download.tensorflow.org/models/object_detection/'
model = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz'
tmp = '/content/checkpoint.tar.gz'

if not (os.path.exists(CHECKPOINT_PATH)):
  # Download the checkpoint
  opener = urllib.request.URLopener()
  opener.retrieve(download_base + model, tmp)

  # Extract all the `model.ckpt` files.
  with tarfile.open(tmp) as tar:
    for member in tar.getmembers():
      member.name = os.path.basename(member.name)
      if 'model.ckpt' in member.name:
        tar.extract(member, path=CHECKPOINT_PATH)

  os.remove(tmp)

## Model config

In [7]:
import re

from google.protobuf import text_format

from object_detection.utils import config_util
from object_detection.utils import label_map_util

pipeline_skeleton = '/content/models/research/object_detection/samples/configs/ssd_mobilenet_v1_quantized_300x300_coco14_sync.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_skeleton)

label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)
num_classes = len(label_map.keys())
meta_arch = configs["model"].WhichOneof("model")

override_dict = {
  'model.{}.num_classes'.format(meta_arch): num_classes,
  'train_config.batch_size': 24,
  'train_input_path': TRAIN_RECORD_PATH,
  'eval_input_path': VAL_RECORD_PATH,
  'train_config.fine_tune_checkpoint': os.path.join(CHECKPOINT_PATH, 'model.ckpt'),
  'label_map_path': LABEL_MAP_PATH
}

configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict)
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, DATA_PATH)


INFO:tensorflow:Maybe overwriting model.ssd.num_classes: 3
INFO:tensorflow:Maybe overwriting train_config.batch_size: 24
INFO:tensorflow:Maybe overwriting train_input_path: /content/data/train.record
INFO:tensorflow:Maybe overwriting eval_input_path: /content/data/val.record
INFO:tensorflow:Maybe overwriting train_config.fine_tune_checkpoint: /content/checkpoint/model.ckpt
INFO:tensorflow:Maybe overwriting label_map_path: /content/data/label_map.pbtxt

INFO:tensorflow:Writing pipeline config file to /content/data/pipeline.config


## Start training

In [8]:
!rm -rf $OUTPUT_PATH
!python -m object_detection.model_main \
    --pipeline_config_path=$DATA_PATH/pipeline.config \
    --model_dir=$OUTPUT_PATH \
    --num_train_steps=$NUM_TRAIN_STEPS \
    --num_eval_steps=100

/content/models/research
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1214 18:50:53.694585 139722179495808 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1214 18:50:53.697578 139722179495808 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W1214 18:50:53.697701 139722179495808 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W1214 18:50:53.697801 139722179495808 module_wrapper.py:139]

## Export frozen graph

In [9]:
import os
import re

regex = re.compile(r"model\.ckpt-([0-9]+)\.index")
numbers = [int(regex.search(f).group(1)) for f in os.listdir(OUTPUT_PATH) if regex.search(f)]
TRAINED_CHECKPOINT_PREFIX = os.path.join(OUTPUT_PATH, 'model.ckpt-{}'.format(max(numbers)))

!python -m object_detection.export_inference_graph \
  --pipeline_config_path=$DATA_PATH/pipeline.config \
  --trained_checkpoint_prefix=$TRAINED_CHECKPOINT_PREFIX \
  --output_directory=$EXPORTED_PATH

/content/models/research
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1214 18:59:51.140379 140046749005696 module_wrapper.py:139] From /content/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1214 18:59:51.148892 140046749005696 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1214 18:59:51.149160 140046749005696 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.c

## Convert the model

In [10]:
!tensorflowjs_converter \
  --input_format=tf_frozen_model \
  --output_format=tfjs_graph_model \
  --output_node_names='Postprocessor/ExpandDims_1,Postprocessor/Slice' \
  --quantization_bytes=1 \
  --skip_op_check \
  $EXPORTED_PATH/frozen_inference_graph.pb \
  /content/model_web

import json

from object_detection.utils.label_map_util import get_label_map_dict

label_map = get_label_map_dict(LABEL_MAP_PATH)
label_array = [k for k in sorted(label_map, key=label_map.get)]

with open(os.path.join('/content/model_web', 'labels.json'), 'w') as f:
  json.dump(label_array, f)

!cd /content/model_web && zip -r /content/model_web.zip *

2019-12-14 19:00:04.675515: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:786] Optimization results for grappler item: graph_to_optimize
2019-12-14 19:00:04.675569: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   debug_stripper: Graph size after: 3226 nodes (0), 3947 edges (0), time = 2.603ms.
2019-12-14 19:00:04.675579: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   model_pruner: Graph size after: 2100 nodes (-1126), 2355 edges (-1592), time = 15.413ms.
2019-12-14 19:00:04.675591: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   constant_folding: Graph size after: 625 nodes (-1475), 687 edges (-1668), time = 214.242ms.
2019-12-14 19:00:04.675602: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   arithmetic_optimizer: Graph size after: 482 nodes (-143), 646 edges (-41), time = 39.66ms.
2019-12-14 19:00:04.675613: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   dependency_optimizer: Graph size after: 476 

## Download the model

In [0]:
from google.colab import files
files.download('/content/model_web.zip') 